Goal: use data provided to find audiences to target with marketing efforts (those who have the highest likelihood of repaying. 
Applicant: Anna Andreyeva 
Date: 2/14/2017 

Data provided by Tala: 
dataAnalystChallenge_people.csv   
dataAnalystChallenge_repayment.csv 
dataAnalystChallenge_device.csv 
dataAnalystChallenge_mobilemoney.csv 

Data Used: 
dataAnalystChallenge_people.csv   
dataAnalystChallenge_repayment.csv 
dataAnalystChallenge_device.csv 

Structure: 
Above CSVs loaded to Tala_phone.ipynb, Tala_People.ipynb, Tala_Repayment.ipynb - data cleaned, modified, output to CSV. 
Cleaned CSVs loaded to Combined_Clean.ipynb - data combined, output to 'merged CSV' 
Merged CSV loaded to Output.ipynb 


Major Assumptions: 
-Borrower is expected to payoff loan within 30 days of origination 
-Only first loans issued were used for analysis, with the assumption that after the first loan, further loans would be based on previous lending experience not the initial set of demographic/phone data
-Dropped loans: those where borrower paid off the loan after 30 days, those that were unpaid but under 30 days outstanding 

Approach: using cross-tabs to pooint out demographic and phone categories where the user had a highest chance of paying off the loan. Onlye variables/categories with 10% and more difference between paid/unpaid loans are considered. 

Results: 

Phone data: Wlan, Bluetooth, Android version 

WLAN - 0.628099 unpaid, 0.778582 paid 
Bluetooth  - 0.461988 unpaid, 0.801837 paid 
Android - the higher the android version, the higher the payoff rate - 0.185185(unknown), 0.490909 (versions 2), 0.788387 (versions 4), 0.952381 (versions 5) 

Demographic: 

Education - higher education levels are associated with higher payoff rate - 0.483146 (primary), 0.816092 (highschool), 0.855346 (college), 0.733333 (masters). Interesting variation between the high school and college (?); decline in the masters education group may be due to lack of data 

Relationship status 'single' (or unknown) is associated with a significantly lower payoff rate than 'married' - 0.879518, 'long-term' - 0.746032 or 'alternative' - 0.875000. 

Friend count does not provide insight into payoff trends. However, the selection of bins may be at fault (bins selected by 1000 friends -> less than 1000, 1000-2000, etc.

Age group data indicates users in higher age groups (30.4 - 43.8 yo & 43.8 -57.2 have significantly higher paayoff rates (0.799528 - 0.862069) than those in the 16.933 - 30.4 age group (0.669683). Insufficient data for users over 57.2 years old. 

Financial/Living Arrangements: 

Number of Years Living at current home data indicates higher number of years is associated with higher loan payoff rates - from 61% payoff rate for those living in a home for under a year or unspecified, to 82.8% for those living in a home for over 5 years. 

Number of Years on the Job - users indicating no years on a current job or those with missing data, have a relatively high payoff rate (0.779116), comparable to those with 5 years on the job (0.764706). Otherwise, payoff rates increase from 0.591837 - less than 1 year on the job to 0.869198 - five years or more on the job. 

Outstanding loan - borrowers with outstanding loans are more likely to payoff loan (0.807018) compared to those without (0.700508) 

Wages data indicate an increased payoff rate for borrower who make over 207,840 per year (0.771635-0.800000), with a payoff rate of 0.663636 for those making under this number. 

Conclcusion: Based on the above-mentioned statistic, TALA should target FB users with the following characteristics: 

Phone - has WLAN, has Bluetooth, has recent version of android 
Demographic - has education at level high school or higher, has relationship status - married, is in age group 43-57 
Financial/Living Arrangements - had been living in a current home for 5 years or more, has been on the job for 5 years or more, has outstanding loan, has wages over 207,840 per year. 

Next Steps: 
1) Evaluation of data assumptions, binning, etc. 
2) Building of logistic regression model to predict probability of payoff given a users demographic inputs based on the conclusions above 


In [463]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [464]:
df = pd.read_csv('final_clean.csv', parse_dates=[0])

In [465]:
df.head(5)

,person_id_random,paidOff,paidOffDuring30,hasDualSim,hasWlan,hasBluetooth,android_version,gender,fb_friend_count,current_home_years,living_situation,current_job_years,education,outstanding_loan,use_mobile_wallet_how_often,relationship_status,Age,job_status,annual_wages
0,7346661357,0,1,0,1,1,4,0,1961,5,0,6,2,0,2,0,25,1,462000.000000
1,9468184407,1,0,0,0,1,4,0,194,5,0,0,2,0,2,1,35,0,352000.000000
2,2990883566,0,1,0,1,1,4,1,445,4,0,6,0,0,1,0,29,1,660000.000000
3,2320175519,1,0,0,1,1,4,0,107,3,0,0,1,0,2,1,46,0,284415.299555
4,5330295937,1,0,0,1,0,2,0,214,5,0,4,0,0,2,0,33,1,284415.299555


In [466]:
pd.crosstab(df.paidOff, df.paidOffDuring30, margins=True)

paidOffDuring30,0,1,All
paidOff,,,
0,30,243,273
1,690,37,727
All,720,280,1000


The data sets provided data on a maximum of 1000 users, broken down as follows: 727 paid off the loans, and 272 have not. 
Of the 727 that paid off the loan, 37 did so after the 1-month deadline. Of the 273 that did not pay off their loans, 30 loans
were within the 1-month payoff deadline at the date the data was generated. This last set of loans will be removed from the data set as it cannot be determined whether they will payoff by the 1-month deadline. 

In [467]:
df=df.drop(df[(df['paidOff']<1) & (df['paidOffDuring30']<1)].index)



Futher, to focus on the users that paid off their loans within the 1-month deadline, data on users who paid off past the deadline will be removed as well. 

In [468]:
df=df.drop(df[(df['paidOff']==1) & (df['paidOffDuring30']==1)].index)
# paidOffDuring30 no longer relevant for categorization
df=df.drop(['paidOffDuring30'],1)

In [469]:
avg_percent_payoff =(len(df[(df['paidOff']==1)])*1.0/len(df))


In [470]:
# create bins for FB count, age & annual wages 
df['friend_count_bins'] = pd.cut(df['fb_friend_count'], 5,)
df['wages_bins'] = pd.cut(df['annual_wages'], 5)
df['age_bins'] = pd.cut(df['Age'],5)
#df['wages_bins'].hist()
#drop original FB count, age & annual wages 
df=df.drop(['fb_friend_count', 'annual_wages', 'Age'],1)

In [471]:
df.head(5)

,person_id_random,paidOff,hasDualSim,hasWlan,hasBluetooth,android_version,gender,current_home_years,living_situation,current_job_years,education,outstanding_loan,use_mobile_wallet_how_often,relationship_status,job_status,friend_count_bins,wages_bins,age_bins
0,7346661357,0,0,1,1,4,0,5,0,6,2,0,2,0,1,"(1000.8, 2000.6]","(395880, 583920]","(16.933, 30.4]"
1,9468184407,1,0,0,1,4,0,5,0,0,2,0,2,1,0,"(-3.999, 1000.8]","(207840, 395880]","(30.4, 43.8]"
2,2990883566,0,0,1,1,4,1,4,0,6,0,0,1,0,1,"(-3.999, 1000.8]","(583920, 771960]","(16.933, 30.4]"
3,2320175519,1,0,1,1,4,0,3,0,0,1,0,2,1,0,"(-3.999, 1000.8]","(207840, 395880]","(43.8, 57.2]"
4,5330295937,1,0,1,0,2,0,5,0,4,0,0,2,0,1,"(-3.999, 1000.8]","(207840, 395880]","(30.4, 43.8]"


In [472]:
pd.crosstab(df.hasDualSim, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
hasDualSim,,
0,0.272237,0.727763
1,0.214660,0.785340


Of users with phones who have dual sim (hasDualSim=1), 78.5% paid off their loans; which is not significantly different from the results for users who do not have a dual sim 72.3% 


In [473]:
pd.crosstab(df.hasWlan, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
hasWlan,,
0,0.371901,0.628099
1,0.221418,0.778582


Of users with phones who have Wlan (hasWlan=1), 77.8% paid off their loans; compared to 62.3% of those who did not

In [474]:
pd.crosstab(df.hasBluetooth, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
hasBluetooth,,
0,0.538012,0.461988
1,0.198163,0.801837


Of users with phones who have Bluetooth (hasBluetooth=1), 80% paid off their loans; compared to 46.2% of those who did not

In [475]:
pd.crosstab(df.android_version, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
android_version,,
0,0.814815,0.185185
2,0.509091,0.490909
4,0.211613,0.788387
5,0.047619,0.952381


Payoff results increase with higher android versions

In [476]:
pd.crosstab(df.gender, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
gender,,
0,0.268945,0.731055
1,0.238462,0.761538


Gender seems irrelevant to loan payoff results

In [477]:
pd.crosstab(df.current_home_years, df.paidOff).apply(lambda r: r/r.sum(), axis=1)



paidOff,0,1
current_home_years,,
0.00000,0.383562,0.616438
1.00000,0.354839,0.645161
2.00000,0.333333,0.666667
3.00000,0.265734,0.734266
3.24218,0.250000,0.750000
4.00000,0.254098,0.745902
5.00000,0.171687,0.828313


Loan payoff rates increase with increased years living at current home

In [478]:
pd.crosstab(df.living_situation, df.paidOff).apply(lambda r: r/r.sum(), axis=1)



paidOff,0,1
living_situation,,
0,0.267185,0.732815
1,0.228395,0.771605


Living situation (rent or own) seems irrelevant to payoff rates. Its possible to try to get more granular data from the set provided. 

In [479]:
pd.crosstab(df.current_job_years, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
current_job_years,,
0,0.220884,0.779116
1,0.408163,0.591837
2,0.378641,0.621359
3,0.401709,0.598291
4,0.333333,0.666667
5,0.235294,0.764706
6,0.130802,0.869198


Surprisingly, users who did not idicate whether they have a job or indicated that they do not have a job, have a high payoff rate (77.9%), comparable to those that have 4 years + on a current job (76% and up) 

In [480]:
pd.crosstab(df.job_status, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
job_status,,
0,0.220884,0.779116
1,0.274854,0.725146


However, job status in and of itself, is not a good indicator whether a borrower is more likely to pay off their loan

In [481]:
pd.crosstab(df.education, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
education,,
0,0.516854,0.483146
1,0.183908,0.816092
2,0.144654,0.855346
3,0.266667,0.733333


Payoff rates increase with education level (jumping from 48% to 82% for those who have high school dimploma versus those who do not. There is a surprising drop in payoff rate at the masters degree level, but that could be due to lack of data (only 15 of the 933 users remaining in the data set indicate they have a masters degree 

In [482]:
pd.crosstab(df.outstanding_loan, df.paidOff).apply(lambda r: r/r.sum(), axis=1)



paidOff,0,1
outstanding_loan,,
0,0.299492,0.700508
1,0.192982,0.807018


People who indicate outstanding loans have a higher payoff percecentage (80.7%) than those who do not (70%) 

In [483]:
pd.crosstab(df.use_mobile_wallet_how_often, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
use_mobile_wallet_how_often,,
0,0.300000,0.700000
1,0.274390,0.725610
2,0.235772,0.764228
3,0.000000,1.000000


Users who indicate frequent use of mobile wallet, are somewhat more likely to pay off their loans. However, the difference in pyaoff rate is not large and more data is required for users in the  10+ times a week category. 

In [484]:
pd.crosstab(df.relationship_status, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
relationship_status,,
0,0.398633,0.601367
1,0.120482,0.879518
2,0.253968,0.746032
3,0.125000,0.875000


Users in relationship status 'married' payoff loans at a significantly higher rate than those who indicate that they are single or do not indicate their relationship status. Those in long-term relationships, also show higher payoff rates (74.6%). 

In [485]:
pd.crosstab(df.friend_count_bins, df.paidOff).apply(lambda r: r/r.sum(), axis=1)


paidOff,0,1
friend_count_bins,,
"(-3.999, 1000.8]",0.250000,0.750000
"(1000.8, 2000.6]",0.280992,0.719008
"(2000.6, 3000.4]",0.347826,0.652174
"(3000.4, 4000.2]",0.416667,0.583333
"(4000.2, 5000]",0.230769,0.769231


In [486]:
pd.crosstab(df.friend_count_bins, df.paidOff, margins=True)

paidOff,0,1,All
friend_count_bins,,,
"(-3.999, 1000.8]",182,546,728
"(1000.8, 2000.6]",34,87,121
"(2000.6, 3000.4]",16,30,46
"(3000.4, 4000.2]",5,7,12
"(4000.2, 5000]",6,20,26
All,243,690,933


User pay off rates do not seem to be influenced by friend counts. Alternatively, the binning break-down does not highlight any underlying trends.

In [487]:
pd.crosstab(df.wages_bins, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
wages_bins,,
"(18859.8, 207840]",0.336364,0.663636
"(207840, 395880]",0.228365,0.771635
"(395880, 583920]",0.191667,0.808333
"(583920, 771960]",0.212766,0.787234
"(771960, 960000]",0.200000,0.800000


In [488]:
pd.crosstab(df.wages_bins, df.paidOff, margins=True)


paidOff,0,1,All
wages_bins,,,
"(18859.8, 207840]",111,219,330
"(207840, 395880]",95,321,416
"(395880, 583920]",23,97,120
"(583920, 771960]",10,37,47
"(771960, 960000]",4,16,20
All,243,690,933


Wage data indicates that users with annual wage of over 207,840 have higher payoff rates (rates fluctuate between 77% and 80% versus 66% for lower wages) 

In [489]:
pd.crosstab(df.age_bins, df.paidOff).apply(lambda r: r/r.sum(), axis=1)

paidOff,0,1
age_bins,,
"(16.933, 30.4]",0.330317,0.669683
"(30.4, 43.8]",0.200472,0.799528
"(43.8, 57.2]",0.137931,0.862069
"(57.2, 70.6]",0.000000,1.000000
"(70.6, 84]",0.666667,0.333333


In [490]:
pd.crosstab(df.age_bins, df.paidOff, margins=True)

paidOff,0,1,All
age_bins,,,
"(16.933, 30.4]",146,296,442
"(30.4, 43.8]",85,339,424
"(43.8, 57.2]",8,50,58
"(57.2, 70.6]",0,3,3
"(70.6, 84]",4,2,6
All,243,690,933


Age group data indicates users in higher age groups (30.4 - 43.8 yo & 43.8 -57.2 have significantly higher paayoff rates (79% - 66%) than those in the 16.933 - 30.4 age group (67%)